In [1]:
import numpy as np
import torch
from dgmc.models import DGMC, RelCNN
from torch_geometric.utils import add_self_loops

from networkx.generators.trees import random_tree
from networkx.generators.random_graphs import erdos_renyi_graph


In [14]:
def corrcoef(x, y):
    
    # calculate covariance matrix of rows
    mean_x = x.mean(dim=0)
    mean_y = y.mean(dim=0)
    xm = x - mean_x
    ym = y - mean_y
    c = xm.T @ ym

    c = c / (x.size(0) - 1)
    cx = (xm ** 2).mean(dim=0).unsqueeze(-1)
    cy = (ym ** 2).mean(dim=0)
    
    

    # normalize covariance matrix
    cxx = torch.sqrt(cx)
    cyy = torch.sqrt(cy)

    c = c / cxx / cyy

    # clamp between -1 and 1
    # probably not necessary but numpy does it
    
#     c = torch.clamp(c, -1.0, 1.0)

    return c

In [15]:
def get_graph(num_nodes, input_dim, p_erdos_renyi, noise=0.1):
    
    ind = np.arange(num_nodes)
    perm = np.random.permutation(ind)
    perm_matrix = torch.eye(num_nodes)[perm, :] + noise * torch.randn(num_nodes, num_nodes) / num_nodes
    perm_matrix = perm_matrix / perm_matrix.sum(axis=-1)
    
    edge_index1 = torch.tensor(list(erdos_renyi_graph(num_nodes, p_erdos_renyi).edges)).T.long()
        
    edge_index1 = add_self_loops(edge_index1)[0]
    
    edge_index2 = torch.tensor(list(map(lambda x: perm[x], edge_index1))).long()
    
    return edge_index1, edge_index2, perm, perm_matrix


def sample_features(num_nodes, input_dim, perm_matrix):
    x1 = torch.randn(num_nodes, input_dim) / input_dim
    x2 = perm_matrix @ x1
    
    return x1, x2

In [16]:
num_nodes = 10
input_dim = 50
p_erdos_renyi = 0.5
noise = 0.3
num_samples = 100


edge_index1, edge_index2, perm, perm_matrix = get_graph(num_nodes, input_dim, p_erdos_renyi, noise=noise)

### Sample Features

In [17]:
model = RelCNN(input_dim, input_dim, num_layers=2)

samples1 = np.empty((num_samples, num_nodes, input_dim))
samples2 = np.empty((num_samples, num_nodes, input_dim))

correlations_np = np.zeros((num_nodes, num_nodes))
correlations_torch = torch.zeros((num_nodes, num_nodes))

with torch.no_grad():
    for i in range(num_samples):
        x1, x2 = sample_features(num_nodes, input_dim, perm_matrix)
        o1, o2 = model(x1, edge_index1), model(x2, edge_index2)
        samples1[i, ...], samples2[i, ...] = o1.numpy(), o2.numpy()
    
for i in range(input_dim):
    
    correlations_np += np.corrcoef(samples1[..., i], samples2[..., i], rowvar=False)[:num_nodes, num_nodes:]
    correlations_torch += corrcoef(torch.tensor(samples1[..., i]), torch.tensor(samples2[..., i]))
    
correlations_np /= input_dim
correlations_torch /= input_dim

In [19]:
correlations_torch

tensor([[ 7.0194e-02,  3.8122e-02,  1.3353e-03,  5.3833e-02,  5.2476e-02,
          4.5994e-02, -2.0446e-02,  9.3536e-01,  2.3990e-02, -1.1133e-02],
        [ 1.0767e-02,  3.0387e-02,  4.0274e-02,  1.2569e-02,  8.0754e-03,
          3.9355e-02, -1.9414e-03,  3.2030e-02,  1.6285e-02,  9.4860e-01],
        [ 2.7664e-02,  1.8018e-04,  2.7334e-02,  8.8295e-02, -6.1076e-02,
         -1.4450e-02,  1.4653e-02,  7.6036e-02,  9.7596e-01, -4.8858e-02],
        [ 8.0868e-02,  5.4823e-02, -2.3413e-02,  6.1443e-02,  1.8944e-03,
          1.1525e-02,  9.8409e-01,  5.6687e-02,  3.1781e-02,  1.9009e-02],
        [ 9.3412e-01,  1.6558e-02,  1.6654e-02,  3.1596e-02,  1.2079e-03,
         -7.8940e-03,  4.1620e-02,  1.6603e-03,  5.8029e-02,  5.2242e-02],
        [-1.5415e-02,  8.3874e-02, -4.6858e-03,  2.7887e-02,  3.7401e-02,
          9.5243e-01,  1.4267e-02,  1.2722e-01, -1.1085e-02,  4.1147e-03],
        [ 2.3283e-02,  9.5193e-01,  3.9119e-02, -1.7490e-02,  4.8298e-02,
          3.8941e-02,  6.7583e-0

In [12]:
correlations_torch

tensor([[ 3.9639e-01,  1.7908e+00,  1.2807e+00,  4.5709e+00,  1.9262e+00,
          7.5582e+00,  2.0229e+00,  9.1736e+01,  3.9169e+00,  9.3683e+00],
        [ 4.2849e+00,  2.6567e+00,  1.5445e+00,  8.9956e+00, -3.7030e+00,
          4.1647e+00,  9.2648e+01,  3.5534e+00,  2.4922e+00,  3.1219e+00],
        [ 8.8043e+00,  3.5335e+00,  9.5729e+01, -8.8903e-01,  5.4679e+00,
          9.3067e+00,  4.1760e+00,  8.5569e+00,  7.1919e+00,  3.7556e+00],
        [ 4.5002e+00,  9.4122e+01,  8.2797e+00, -2.7030e+00,  1.3039e+01,
         -7.9638e-02,  3.3222e+00,  7.8999e+00,  6.5382e+00,  9.6000e-01],
        [ 3.5989e+00,  4.4004e+00,  4.8145e+00,  2.6932e+00,  1.1179e+00,
          2.7437e+00,  1.0255e+00,  1.0694e+01,  9.6609e+01,  6.6346e+00],
        [ 1.0100e+01,  1.2057e+00,  3.6389e+00,  4.2708e+00,  5.9428e+00,
          9.4177e+01,  5.5745e+00, -1.7516e+00,  6.5628e+00,  2.4937e+00],
        [ 9.3881e+01,  6.5884e+00,  6.0595e+00,  1.9447e+00, -6.5913e+00,
          1.6132e+00,  8.5254e+0

In [188]:
correlations.argmax(axis=0) - perm_matrix.argmax(dim=-1).numpy()

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### Sample models

In [189]:
x1, x2 = sample_features(num_nodes, input_dim, perm_matrix)

samples1 = np.empty((num_samples, num_nodes, input_dim))
samples2 = np.empty((num_samples, num_nodes, input_dim))

correlations = np.zeros((num_nodes, num_nodes))

with torch.no_grad():
    for i in range(num_samples):
        model = RelCNN(input_dim, input_dim, num_layers=2)
        o1, o2 = model(x1, edge_index1), model(x2, edge_index2)
        samples1[i, ...], samples2[i, ...] = o1.numpy(), o2.numpy()
    
for i in range(input_dim):
    correlations += np.corrcoef(samples1[..., i], samples2[..., i], rowvar=False)[:num_nodes, num_nodes:]
    
correlations /= input_dim

In [190]:
correlations.argmax(axis=0) - perm_matrix.argmax(dim=-1).numpy()

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [86]:
samples1[..., i].shape

(100, 10)

In [101]:
samples1[..., i]

array([[0.45470685, 0.2832554 , 0.19715106, 0.26954108, 0.25679341,
        0.2262544 , 0.31731504, 0.27620322, 0.24867499, 0.27105612],
       [0.45470685, 0.2832554 , 0.19715106, 0.26954108, 0.25679341,
        0.2262544 , 0.31731504, 0.27620322, 0.24867499, 0.27105612],
       [0.45470685, 0.2832554 , 0.19715106, 0.26954108, 0.25679341,
        0.2262544 , 0.31731504, 0.27620322, 0.24867499, 0.27105612],
       [0.45470685, 0.2832554 , 0.19715106, 0.26954108, 0.25679341,
        0.2262544 , 0.31731504, 0.27620322, 0.24867499, 0.27105612],
       [0.45470685, 0.2832554 , 0.19715106, 0.26954108, 0.25679341,
        0.2262544 , 0.31731504, 0.27620322, 0.24867499, 0.27105612],
       [0.45470685, 0.2832554 , 0.19715106, 0.26954108, 0.25679341,
        0.2262544 , 0.31731504, 0.27620322, 0.24867499, 0.27105612],
       [0.45470685, 0.2832554 , 0.19715106, 0.26954108, 0.25679341,
        0.2262544 , 0.31731504, 0.27620322, 0.24867499, 0.27105612],
       [0.45470685, 0.2832554 , 0.1971510

In [105]:
S = np.corrcoef(samples1[..., i], samples2[..., i], rowvar=False)